In [2]:
import csv
import json
import pandas as pd

# Load dummy_pooling.csv
dummy_pooling_file = 'Pooling_Output/sorted_pooling_jan_31_all_trips.csv'
dummy_pooling = pd.read_csv(dummy_pooling_file)

# Load dummy_traj.json
dummy_traj_file = 'original_traj_mapped.json'
dummy_traj = []

with open(dummy_traj_file, 'r') as f:
    for line in f:
        try:
            trip = json.loads(line.strip())
            dummy_traj.append(trip)
        except json.JSONDecodeError:
            continue  # Skip lines that are not valid JSON

# Extract trip_ids from dummy_traj
traj_trip_ids = {trip['trip_id'] for trip in dummy_traj}

# Extract booking_ids from dummy_pooling
pooling_trip_ids = set(dummy_pooling['booking_id'])

# Trips in dummy_pooling but not in dummy_traj
pooling_not_in_traj = dummy_pooling[~dummy_pooling['booking_id'].isin(traj_trip_ids)]
pooling_not_in_traj_file = 'pooling_not_in_traj.csv'
pooling_not_in_traj.to_csv(pooling_not_in_traj_file, index=False)

# Trips in dummy_traj but not in dummy_pooling
traj_not_in_pooling = [trip for trip in dummy_traj if trip['trip_id'] not in pooling_trip_ids]
traj_not_in_pooling_file = 'traj_not_in_pooling.csv'

# Save traj_not_in_pooling to CSV
with open(traj_not_in_pooling_file, 'w', newline='') as f:
    if traj_not_in_pooling:
        writer = csv.DictWriter(f, fieldnames=traj_not_in_pooling[0].keys())
        writer.writeheader()
        writer.writerows(traj_not_in_pooling)

print(f'CSV file created for trips in pooling but not in traj: {pooling_not_in_traj_file}')
print(f'CSV file created for trips in traj but not in pooling: {traj_not_in_pooling_file}')


CSV file created for trips in pooling but not in traj: pooling_not_in_traj.csv
CSV file created for trips in traj but not in pooling: traj_not_in_pooling.csv
